In [19]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats
import pandas as pd

In [101]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [24]:
#low-variability
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
# correlation_filtering()

# se define la funcion
def correlation_filtering(data_frame, treshold):

    df = pd.DataFrame(data_frame)
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # guarda las columnas que se van a tirar
    to_drop =[]

    for n in range(df.shape[1]):
        print(n)
        if any(upper[n]>treshold):
            print("La columna ", n, "no pasa el umbral")
            to_drop.append(df.columns[n])

    print(to_drop)        

    # Drop features 
    df.drop(df.columns[to_drop], axis=1)

    return df

In [33]:
correlation_filtering(X, 0.9)

0
1
2
3
4
5
6
7
8
9
La columna  9 no pasa el umbral
10
11
12
[9]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [102]:
# Fast Correlation Based filtering
dfx = pd.DataFrame(X)
#target = pd.DataFrame(y)
# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.6
corr_xy = []
high_corr = []

for n in range(dfx.shape[1]):
    column = dfx[n]
    a = abs(np.corrcoef(column, y)[0,1])
    corr_xy.append(a)
    if a > treshold:
        high_corr.append(dfx.columns[n])
    
print(corr_xy) 
print(high_corr)

# 2. Ordenar las variables de mayor a menor correlación con Y
temp = pd.DataFrame(corr_xy).reset_index()
temp = temp.rename(columns={0: 'corr'})
temp = temp.sort_values(by=['corr'], ascending=False)
print(temp)             
#column1 = dfx[0]
#column2 = dfx[12]
#column2.corr(column1, method='pearson')
#a = abs(np.corrcoef(column1, y)[0,1])

#b = scipy.stats.pearsonr(column1, y)[0]
#print(a, b)

[0.38830460858681143, 0.3604453424505435, 0.48372516002837357, 0.1752601771902987, 0.4273207723732826, 0.6953599470715393, 0.37695456500459623, 0.24992873408590394, 0.3816262306397775, 0.46853593356776685, 0.5077866855375616, 0.3334608196570666, 0.7376627261740151]
[5, 12]
    index      corr
12     12  0.737663
5       5  0.695360
10     10  0.507787
2       2  0.483725
9       9  0.468536
4       4  0.427321
0       0  0.388305
8       8  0.381626
6       6  0.376955
1       1  0.360445
11     11  0.333461
7       7  0.249929
3       3  0.175260


In [103]:
# 3. Medir la correlación de la mejor variable con las demás

# Create correlation matrix
corr_matrix = dfx.corr().abs()

# elementos en la diagonal se vuelven 0
corr_matrix.values[[np.arange(corr_matrix.shape[0])]*2] = 0
# print(corr_matrix)

          0         1         2         3         4         5         6   \
0   0.000000  0.200469  0.406583  0.055892  0.420972  0.219247  0.352734   
1   0.200469  0.000000  0.533828  0.042697  0.516604  0.311991  0.569537   
2   0.406583  0.533828  0.000000  0.062938  0.763651  0.391676  0.644779   
3   0.055892  0.042697  0.062938  0.000000  0.091203  0.091251  0.086518   
4   0.420972  0.516604  0.763651  0.091203  0.000000  0.302188  0.731470   
5   0.219247  0.311991  0.391676  0.091251  0.302188  0.000000  0.240265   
6   0.352734  0.569537  0.644779  0.086518  0.731470  0.240265  0.000000   
7   0.379670  0.664408  0.708027  0.099176  0.769230  0.205246  0.747881   
8   0.625505  0.311948  0.595129  0.007368  0.611441  0.209847  0.456022   
9   0.582764  0.314563  0.720760  0.035587  0.668023  0.292048  0.506456   
10  0.289946  0.391679  0.383248  0.121515  0.188933  0.355501  0.261515   
11  0.385064  0.175520  0.356977  0.048788  0.380051  0.128069  0.273534   
12  0.455621

<ipython-input-103-186e5df949f6>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  corr_matrix.values[[np.arange(corr_matrix.shape[0])]*2] = 0


In [126]:
to_drop = []
for index, row in temp.iterrows():
    if row['corr']>treshold:     
#        print(row['corr'])
#        print(row['index'])
#        print(index)
#        print(corr_matrix.values[index,0])
        for column in range(corr_matrix.shape[1]):
#            print(corr_matrix.values[index,column])
            if corr_matrix.values[index,column] > treshold:
                print("Row ", index, "Column ", column, "valor ", corr_matrix.values[index,column])
                to_drop.append(dfx.columns[column])
        break    

print(to_drop)

# 4. Tirar variables que estén muy correlacionadas con la primera variable.
new_dfx = dfx.drop(dfx.columns[to_drop], axis=1)
print(new_dfx.head())

Row  12 Column  2 valor  0.6037997164766213
Row  12 Column  5 valor  0.6138082718663957
Row  12 Column  6 valor  0.6023385287262399
[2, 5, 6]
        0     1    3      4       7    8      9     10      11    12
0  0.00632  18.0  0.0  0.538  4.0900  1.0  296.0  15.3  396.90  4.98
1  0.02731   0.0  0.0  0.469  4.9671  2.0  242.0  17.8  396.90  9.14
2  0.02729   0.0  0.0  0.469  4.9671  2.0  242.0  17.8  392.83  4.03
3  0.03237   0.0  0.0  0.458  6.0622  3.0  222.0  18.7  394.63  2.94
4  0.06905   0.0  0.0  0.458  6.0622  3.0  222.0  18.7  396.90  5.33


In [ ]:
# FCB_filtering()
# def FCB_filtering(X,Y,treshold):
    '''
    Función que elige características de una base de datos con base en la correlación de las características 
    con la variable objetivo.
    Inputs:
    X: características
    Y: variable objetivo
    treshold: umbral de correlación
    
    Nota: el algoritmo se basa en Yu & Liu (2003) y sólo itera sobre las características relevantes,
    es decir las que están por encima del treshold definido.
    '''
    
    
    
